In [11]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [12]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [13]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [7]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

Download the source code for PELCO I's webpage.

In [20]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests

# Folder to store the downloaded files
download_folder = "pdf downloads"
os.makedirs(download_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Function to extract year-month codes from the dynamically loaded HTML
def extract_year_month_codes(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    driver.get(url)
    driver.implicitly_wait(10)  # Wait for elements to load

    year_month_codes = []

    # Locate rows with "Generation Rates"
    try:
        rows = driver.find_elements(By.CSS_SELECTOR, 'div.w-full.grid.grid-cols-5.border-b.border-gray-200')
        for row in rows:
            headers = row.find_elements(By.CSS_SELECTOR, 'div.flex.items-center.justify-start.px-4.py-2')
            if len(headers) >= 3:
                # Extract text values
                month_text = headers[0].find_element(By.TAG_NAME, 'h6').text.strip()
                year_text = headers[1].find_element(By.TAG_NAME, 'h6').text.strip()
                rate_text = headers[2].find_element(By.TAG_NAME, 'h6').text.strip()

                if "Generation Rates" in rate_text:
                    month_map = {
                        "January": "01", "February": "02", "March": "03",
                        "April": "04", "May": "05", "June": "06",
                        "July": "07", "August": "08", "September": "09",
                        "October": "10", "November": "11", "December": "12"
                    }
                    if month_text in month_map:
                        year_month_codes.append(f"{year_text}{month_map[month_text]}")
    except Exception as e:
        print(f"An error occurred: {e}")

    driver.quit()
    return year_month_codes

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    if response.status_code == 200:
        file_path = os.path.join(download_folder, file_name)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download: {file_name}")

# Main
rates_url = "https://www.pelco1.org.ph/rates"
base_url = "https://firebasestorage.googleapis.com/v0/b/pelco1-cms.appspot.com/o/rates%2Fgen_"
file_extension = ".pdf"

# Extract year-month codes and download files
year_month_list = extract_year_month_codes(rates_url)
for ym in year_month_list:
    file_name = f"gen_{ym}.pdf"
    url = f"{base_url}{ym}{file_extension}?alt=media"
    download_file(url, file_name)

Downloaded: gen_202401.pdf
Downloaded: gen_202402.pdf
Downloaded: gen_202403.pdf
Downloaded: gen_202404.pdf
Downloaded: gen_202405.pdf
Downloaded: gen_202406.pdf
Downloaded: gen_202407.pdf
Failed to download: gen_202408.pdf
Downloaded: gen_202301.pdf
Downloaded: gen_202302.pdf
Downloaded: gen_202303.pdf
Downloaded: gen_202304.pdf
Downloaded: gen_202305.pdf
Downloaded: gen_202306.pdf
Downloaded: gen_202307.pdf
Downloaded: gen_202308.pdf
Downloaded: gen_202309.pdf
Downloaded: gen_202310.pdf
Downloaded: gen_202311.pdf
Downloaded: gen_202312.pdf
Downloaded: gen_202201.pdf
Downloaded: gen_202202.pdf
Downloaded: gen_202203.pdf
Downloaded: gen_202204.pdf
Downloaded: gen_202205.pdf
Downloaded: gen_202206.pdf
Downloaded: gen_202207.pdf
Downloaded: gen_202208.pdf
Downloaded: gen_202209.pdf
Downloaded: gen_202210.pdf
Downloaded: gen_202211.pdf
Downloaded: gen_202212.pdf
Downloaded: gen_202101.pdf
Downloaded: gen_202102.pdf
Downloaded: gen_202103.pdf
Downloaded: gen_202104.pdf
Downloaded: gen_2021

### **Processing**

#### troubleshooting loop

In [147]:
df = tabula.read_pdf("pdf downloads/gen_201907.pdf", stream=True, pages=1)[0]

In [148]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,(A),(B),Unnamed: 3,( C ),(D = B+C),(D/A)
0,NaN,SOURCE,% To Total,NaN,Basic,NaN,Other Cost,Total,Average
1,NaN,NaN,Kwh,Kwh,Generation,NaN,Adjustments,Generation,Generation
2,NaN,NaN,Purchased,Purchased,Cost (Php),NaN,"(DAA, others)",Cost for the,Cost
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,month (Php),(Php/kwh)
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AES-TSC,NaN,100.00%,"22,692,998","128,352,926.47",-,NaN,"128,352,926.47",5.6561
6,Bilateral Contracts with IPPs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,WESM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
# Define the keywords to search for in row values
keywords = ['CONTRACTS', ', ', '.']

# Initialize a list to store the columns to keep
columns_to_keep = []

# First column to keep: Find the column containing the word 'CONTRACTS'
contracts_col = df.columns[df.apply(lambda col: col.astype(str).str.contains('CONTRACTS', case=False, na=False)).any()]
if not contracts_col.empty:
    contracts_col_name = contracts_col[0]
    columns_to_keep.append(contracts_col_name)

    # Second column to keep: Find the first column with a comma, excluding the 'CONTRACTS' column
    comma_col = df.columns[
        df.apply(lambda col: col.astype(str).str.contains(',', case=False, na=False)).any()
    ]
    if not comma_col.empty:
        comma_col_name = comma_col[0]
        
        if comma_col_name != contracts_col_name:
            columns_to_keep.append(comma_col_name)
        else:
            comma_col_name = comma_col[1]
            columns_to_keep.append(comma_col_name)

        # Third column to keep: Find the last column with a period, excluding the 'comma_col' column
        period_cols = df.columns[
            df.apply(lambda col: col.astype(str).str.contains('.', case=False, na=False)).any()
        ]
        
        if not period_cols.empty:
            period_cols = [col for col in period_cols if col != comma_col_name]
            if period_cols:
                last_period_col = period_cols[-1]
                if last_period_col not in columns_to_keep:
                    columns_to_keep.append(last_period_col)

# Drop duplicates to ensure unique columns
columns_to_keep = list(dict.fromkeys(columns_to_keep))

# Slice the DataFrame to keep only the relevant columns
df_sliced_columns = df[columns_to_keep]

# Rename columns for consistency if the length matches
if len(df_sliced_columns.columns) == 3:
    df_sliced_columns.columns = ["Power Supplier", "kWh", "Average Generation Cost"]

df_sliced_columns

,Power Supplier,kWh,Average Generation Cost
0,NaN,NaN,Average
1,NaN,Kwh,Generation
2,NaN,Purchased,Cost
3,NaN,NaN,(Php/kwh)
4,NaN,NaN,NaN
5,AES-TSC,"22,692,998",5.6561
6,Bilateral Contracts with IPPs,NaN,NaN
7,1.,NaN,NaN
8,2.,NaN,NaN
9,WESM,NaN,NaN


In [145]:
# Check for leading numbers in the first column (assuming 'Power Supplier' is the first column here)
df_filtered_leading_numbers = df_sliced_columns[df_sliced_columns['Power Supplier'].str.contains(r'^\d+', na=False)]

# Filter rows based on whether any column contains 'WESM'
df_filtered_keywords = df_sliced_columns[df_sliced_columns.apply(
    lambda row: row.astype(str).str.contains('WESM', case=False, na=False).any(), axis=1
)]

# Combine both filters
df_new = pd.concat([df_filtered_keywords, df_filtered_leading_numbers]).drop_duplicates().copy()

# Remove leading numbers and replace \r with space in the "Power Supplier" column values
if 'Power Supplier' in df_new.columns:
    df_new['Power Supplier'] = df_new['Power Supplier'].apply(
        lambda x: re.sub(r'^\d+\.\s*', '', str(x)).replace('\r', ' ')
    )

# Reset the index of the final dataframe
df_new.reset_index(drop=True, inplace=True)

df_new

,Power Supplier,kWh,Average Generation Cost
0,"Bac-Man Geothermal, Inc. (BGI)","1 0,800,000",5 .1978
1,Wholesale Electricity Spot Market,"1 9,304,510",8 .9122
2,Net-Metering,"3 1,858",7 .4968


In [126]:
# Find the row with 'TOTAL' and extract the value from 'Column Average'
total_row = df_sliced_columns[df_sliced_columns['Power Supplier'].str.contains('TOTAL', case=False, na=False)]
generation_charge = total_row['Average Generation Cost'].values[0] if not total_row.empty else None

# Add the new column "Generation Charge" with the extracted value
df_new['Generation Charge'] = generation_charge

# Reset the index of the final dataframe
df_new.reset_index(drop=True, inplace=True)

df_new

,Power Supplier,kWh,Average Generation Cost,Generation Charge
0,WESM,NaN,NaN,5.4182
1,MPPCL,"25,586,985",5.4182,5.4182


#### looped

In [154]:
import os
import tabula
import pandas as pd
import re

# Initialize an empty list to store the dataframes
all_dfs = []
error_files = []

# Define the path to the folder containing the PDFs
pdf_folder = 'pdf downloads'

# Iterate over each file in the folder
for file_name in os.listdir(pdf_folder):
    if file_name.endswith('.pdf'):
        file_path = os.path.join(pdf_folder, file_name)
        
        try:
            # Read the PDF file
            df = tabula.read_pdf(file_path, stream=True, pages=1)[0]
            
            # Define the keywords to search for in row values
            keywords = ['CONTRACTS', ', ', '.']
            
            # Initialize a list to store the columns to keep
            columns_to_keep = []
            
            # First column to keep: Find the column containing the word 'CONTRACTS'
            contracts_col = df.columns[df.apply(lambda col: col.astype(str).str.contains('CONTRACTS', case=False, na=False)).any()]
            if not contracts_col.empty:
                contracts_col_name = contracts_col[0]
                columns_to_keep.append(contracts_col_name)
                
                # Second column to keep: Find the first column with a comma, excluding the 'CONTRACTS' column
                comma_col = df.columns[
                    df.apply(lambda col: col.astype(str).str.contains(',', case=False, na=False)).any()
                ]
                if not comma_col.empty:
                    comma_col_name = comma_col[0]
                    
                    if comma_col_name != contracts_col_name:
                        columns_to_keep.append(comma_col_name)
                    else:
                        # Handle case where the comma column is the same as 'CONTRACTS'
                        if len(comma_col) > 1:
                            comma_col_name = comma_col[1]
                            columns_to_keep.append(comma_col_name)
                        
                # Third column to keep: Find the last column with a period, excluding the 'comma_col' column
                period_cols = df.columns[
                    df.apply(lambda col: col.astype(str).str.contains('.', case=False, na=False)).any()
                ]
                if not period_cols.empty:
                    period_cols = [col for col in period_cols if col != comma_col_name]
                    if period_cols:
                        last_period_col = period_cols[-1]
                        if last_period_col not in columns_to_keep:
                            columns_to_keep.append(last_period_col)
            
            # Drop duplicates to ensure unique columns
            columns_to_keep = list(dict.fromkeys(columns_to_keep))
            
            # Slice the DataFrame to keep only the relevant columns
            df_sliced_columns = df[columns_to_keep]
            
            # Rename columns for consistency if the length matches
            if len(df_sliced_columns.columns) == 3:
                df_sliced_columns.columns = ["Power Supplier", "kWh", "Average Generation Cost"]
            
            # Check for leading numbers in the first column (assuming 'Power Supplier' is the first column here)
            df_filtered_leading_numbers = df_sliced_columns[df_sliced_columns['Power Supplier'].str.contains(r'^\d+', na=False)]
            
            # Filter rows based on whether any column contains 'WESM'
            df_filtered_keywords = df_sliced_columns[df_sliced_columns.apply(
                lambda row: row.astype(str).str.contains('WESM', case=False, na=False).any(), axis=1
            )]
            
            # Combine both filters
            df_new = pd.concat([df_filtered_keywords, df_filtered_leading_numbers]).drop_duplicates().copy()
            
            # Remove leading numbers and replace \r with space in the "Power Supplier" column values
            if 'Power Supplier' in df_new.columns:
                df_new['Power Supplier'] = df_new['Power Supplier'].apply(
                    lambda x: re.sub(r'^\d+\.\s*', '', str(x)).replace('\r', ' ')
                )
            
            # Reset the index of the final dataframe
            df_new.reset_index(drop=True, inplace=True)
            
            # Find the row with 'TOTAL' and extract the value from 'Average Generation Cost'
            total_row = df_sliced_columns[df_sliced_columns['Power Supplier'].str.contains('TOTAL', case=False, na=False)]
            generation_charge = total_row['Average Generation Cost'].values[0] if not total_row.empty else None
            
            # Add the new column "Generation Charge" with the extracted value
            df_new['Generation Charge'] = generation_charge
            
            # Reset the index of the final dataframe
            df_new.reset_index(drop=True, inplace=True)
            
            # Append the dataframe to the list
            all_dfs.append(df_new)
        
        except Exception as e:
            # Log the file that caused the error and the exception message
            error_files.append((file_name, str(e)))
            print(f"Error processing file {file_name}: {e}")

# Concatenate all dataframes into a single dataframe
if all_dfs:
    big_df = pd.concat(all_dfs, ignore_index=True)
else:
    big_df = pd.DataFrame()  # If no valid files were processed, create an empty DataFrame

# Print the files that caused errors
if error_files:
    print("Errors occurred with the following files:")
    for file_name, error_message in error_files:
        print(f"File: {file_name}, Error: {error_message}")

big_df


,Power Supplier,kWh,Average Generation Cost,Generation Charge
0,WESM,NaN,NaN,5.3217
1,MPPCL,"27,792,443",5.3217,5.3217
2,WESM,NaN,NaN,5.6735
3,MPPCL,"24,507,511",5.6735,5.6735
4,WESM,NaN,NaN,4.2312
5,MPPCL,"24,870,949",4.2312,4.2312
6,WESM,NaN,NaN,4.2578
7,MPPCL,"24,268,021",4.2578,4.2578
8,"Bac-Man Geothermal, Inc. (BGI)","1 0,800,000",5 .1978,7 .5796
9,Wholesale Electricity Spot Market,"1 9,304,510",8 .9122,7 .5796
